<h3>Exercise 12 - 6 axis Anthropomorphic Robot UR5 with Claw Manipulator with voice recognition & Yolov5</h3>


Contents: ARM & CLAW Tip Manipulator Control, Pick & Place & Rotation - 6-Axis Anthropomorphic Robot, Inverse Kinematics by Coppelia, Image Recognition by YOLO5v, speech recognition & Translation algorithm implementation

To carry out the exercise it is necessary to install a series of libraries that will be defined in each section <br>

We also need the scene that:
<img src="ur5.png" width="500"><br>

# SECTION I  -  INVERSE KINEMATICS ON A UNIVERSAL ROBOT - UR5

### WE DEFINE THE FUNCTIONS THAT WILL ALLOW US TO CONNECT WITH THE ROBOT AND OBJECTS IN THE SCENE

In [ ]:
class CoppeliaSim:
    clientId = 0

    def __init__(self):
        self.clientID = 0

    def connect(self, port):
        sim.simxFinish(-1)
        self.clientID = sim.simxStart('127.0.0.1', port, True, True, 5000, 5)
        CoppeliaSim.clientId = self.clientID
        if self.clientID != -1:
            sim.simxStartSimulation(self.clientID, sim.simx_opmode_blocking)
            print('Connected to remote API server')
        else:
            print('Failed connecting to remote API server')

    def get_object_handle(self, obj_name):
        res, handle = sim.simxGetObjectHandle(self.clientID, obj_name, sim.simx_opmode_blocking)
        return handle
    def get_image(self, handlenum):   
        retCode, resolution, image = sim.simxGetVisionSensorImage(self.clientID,handlenum,0,sim.simx_opmode_oneshot_wait)
        return image,resolution


### WE DEFINE THE FUNCTIONS THAT WILL ALLOW US TO MOVE THE ROBOT WITH THE INVERSE KINEMATICS IMPLEMENTED BY COPPELIASIM

In [ ]:
class Robot(CoppeliaSim):

    def __init__(self, robot_name):
        self.clientID = CoppeliaSim.clientId
        self.robot_name = robot_name
        self.targetName = "./" + robot_name + '/ikTarget'
        self.ftSensorName = './' + robot_name + '/force_sensor'

        # Retrieve object handle
        res, self.robot_handle = sim.simxGetObjectHandle(self.clientID, robot_name, sim.simx_opmode_blocking)
        res, self.target_handle = sim.simxGetObjectHandle(self.clientID, self.targetName, sim.simx_opmode_blocking)
        res, self.ftSensor_handle = sim.simxGetObjectHandle(self.clientID, self.ftSensorName, sim.simx_opmode_blocking)

        # Start position data streaming
        self.script = '/' + robot_name
        sim.simxCallScriptFunction(self.clientID, self.script,
                                   sim.sim_scripttype_childscript,
                                   'remoteApi_getPosition',
                                   [], [], [], '',
                                   sim.simx_opmode_streaming)
        # Moving status data streaming
        sim.simxGetInt32Signal(self.clientID, 'moving_status', sim.simx_opmode_streaming)
        sim.simxGetStringSignal(self.clientID, 'test_signal', sim.simx_opmode_streaming)

        time.sleep(0.05)

    def read_object_handle(self):
        print('Robot handle  = %d' % self.robot_handle)
        print('Target handle = %d' % self.target_handle)

    def get_object_position(self, obj_name):
        res, handle = sim.simxGetObjectHandle(self.clientID, obj_name, sim.simx_opmode_blocking)
        res, pos = sim.simxGetObjectPosition(self.clientID, handle, self.robot_handle, sim.simx_opmode_oneshot_wait)
        res, ori = sim.simxGetObjectOrientation(self.clientID, handle, self.robot_handle, sim.simx_opmode_oneshot_wait)
        ret = [0, 0, 0, 0, 0, 0]
        for i in range(3):
            ret[i] = pos[i] * 1000
            ret[i + 3] = ori[i] * 180 / 3.14
        return ret

    def read_position(self):
        ret = sim.simxCallScriptFunction(self.clientID, self.script,
                                         sim.sim_scripttype_childscript,
                                         'remoteApi_getPosition',
                                         [], [], [], '',
                                         sim.simx_opmode_buffer)
        posData = ret[2]
        for i in range(3):
            posData[i] = posData[i] * 1000
            posData[i + 3] = posData[i + 3] * 180 / 3.14
        return posData

    def set_position(self, pos):
        cmdPos = [0, 0, 0, 0, 0, 0]
        for i in range(3):
            cmdPos[i] = pos[i] / 1000
            cmdPos[i + 3] = pos[i + 3] * 3.141592 / 180

        ret = sim.simxCallScriptFunction(self.clientID, self.script,
                                         sim.sim_scripttype_childscript,
                                         'remoteApi_movePosition',
                                         [], cmdPos, [], '',
                                         sim.simx_opmode_blocking)

    def set_position2(self, pos, wait):
        self.set_position(pos)
        if wait:
            while True:
                time.sleep(0.01)
                if self.isMoving() == 'NOT_MOVING':
                    break

    # Check whether the robot is moving
    def isMoving(self):
        ret, s = sim.simxGetStringSignal(self.clientID, 'test_signal', sim.simx_opmode_buffer)
        s = s.decode('ascii')
        return s

    # Set Robot Speed: 0 - 100
    def set_speed(self, velocity):
        ret = sim.simxCallScriptFunction(self.clientID, self.script,
                                         sim.sim_scripttype_childscript,
                                         'remoteApi_setSpeed',
                                         velocity, [], [], '',
                                         sim.simx_opmode_blocking)

    # val is Int with a value of 0 or 1 to disable the claw     
    def gripper(self, val):
        res=sim.simxCallScriptFunction(self.clientID,"ROBOTIQ_85",
                                       sim.sim_scripttype_childscript,
                                       "gripper",[val],[],[],"",
                                       sim.simx_opmode_blocking)
        

### WE START CONNECTION WITH THE SCENE (REMEMBER BEFORE PLAYING THE SCENE)

In [ ]:
import time
import sim

ur5 = CoppeliaSim()
ur5.connect(19999)

robot = Robot('UR5')

#### WE CAN KNOW IN WHAT POSITION AND ORIENTATION THE FINAL EFFECTOR IS

In [ ]:
pos = robot.read_position()
print('Initial Robot Position: ', pos)

#### WE MOVE USING REVERSE KINEMATICS TO THREE POSITIONS, THE PATH IS DONE IN A STRAIGHT LINE

In [ ]:
# pos1 = [500, 0, 200, 180, 0, 90]
# pos2 = [500, 500, 200, 180, 0, 90]
# pos3 = [0, 500, 200, 180, 0, 90]
# robot.set_position2(pos1, True)
# robot.set_position2(pos2, True)
# robot.set_position2(pos3, True)
# robot.set_position2(pos1, True)

#### WE CAN MOVE THE MANIPULATOR AND OBSERVE ITS BEHAVIOR IN THE X AXIS

In [ ]:
# pos4 = [500, 0, 200, 150, 0, 90]
# pos5 = [500, 0, 200, 210, 0, 90]
# robot.set_position2(pos4, True)
# # robot.set_position2(pos5, True)
# robot.set_position2(pos1, True)

#### Y AXIS

In [ ]:
# pos6 = [500, 0, 200, 180, 30, 90]
# pos7 = [500, 0, 200, 180, -30, 90]
# robot.set_position2(pos6, True)
# robot.set_position2(pos7, True)
# robot.set_position2(pos1, True)

#### Z AXIS

In [ ]:
# pos8 = [500, 0, 200, 180, 0, 50]
# pos9 = [500, 0, 200, 180, 0, 140]
# robot.set_position2(pos8, True)
# robot.set_position2(pos9, True)
# robot.set_position2(pos1, True)

#### WITH THE GRIPPER FUNCTION WE CAN CLOSE AND OPEN THE CLAMPS

In [ ]:
# robot.gripper(1) # CLOSE

In [ ]:
# robot.gripper(0) # OPEN

#### WITH THE GET_OBJECT_POSITION FUNCTION WE CAN KNOW IN WHAT POSITION AND ORIENTATION THE PIECES ARE ON THE TABLE

In [ ]:
# BoxPosition=robot.get_object_position( "Box")
# print(BoxPosition) 

#### WE PERFORM A FUNCTION TO PICK UP AND MOVE THE BOX

In [ ]:
def move_box():
    pos1 = [500, 0, 200, 180, 0, 90]
    pos10 = [500, 0, -10, 180, 0, 90]  # # POSITION OF THE BOX TO CARRY OUT THE PICK
    robot.set_position2(pos1, True)
    robot.gripper(0)
    time.sleep(2)
    robot.set_position2(pos10, True)
    time.sleep(2)
    robot.gripper(1)
    time.sleep(2)
    robot.set_position2(pos1, True)

In [ ]:
# move_box()

### NOTE: we can see that he is able to pick up the box, although for a few seconds since this type of object has flat surfacess

# SECTION II  -  RECOGNITION OF OBJECTS ON THE TABLE


## Practical application of Artificial Intelligence Yolo5v

We will use a camera and the Yolo5v to know what objects are on the table

This program is made in a Jupyter notebook and programmed in python
Requires installing the following libraries and programs from Anaconda:

From Anaconda Terminal:

- Step 1 - installing OpenCV
pip install opencv-python
python
import cv2

- Step 2 - Check
OpenCV Version
cv2.version
exit()

In case of failure, look at https://programarfacil.com/blog/vision-artificial/instalar-opencv-python-anaconda/

- Step 3 - Installing YOLOv5 (requires Python>=3.7.0 and PyTorch>=1.7.

pip install yolov5
cd yolov5
pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt
sudo pip install scikit-learn


In [ ]:
import torch
import pandas
import time
import re

import cv2 # opencv
from matplotlib import pyplot as plt


import numpy as np
from sympy import *

# %matplotlib inline

#### This code loads a pre-trained YOLOv5s model from PyTorch Hub as a model and passes an image to an already trained yolov5s database (it is the lightest and fastest image and data recognition model).

In [ ]:
# acquisition of the Yolo5v Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # o yolov5n - yolov5x6...

#### GET THE HANDLE OF THE CAMERA

In [ ]:
camera=ur5.get_object_handle('Vision_sensor')
print(camera)

THE IMAGES THAT APPEAR IN THE COPPELIA HAVE BEEN MADE USING TEXTURES AND AN ONLINE PROGRAM HAS BEEN USED FOR THEIR TREATMENT:
Application textures images 256x 256 https://imagen.online-convert.com/es/convertir-a-gif

In [ ]:
# We obtain the image from the Camera Sensor
image,resolution=ur5.get_image(camera)
img=np.array(image).astype(np.uint8)
img.resize([resolution[1],resolution[0],3])
img = img[::-1, ::-1, :]

In [ ]:
from PIL import Image
import IPython.display as display

In [ ]:
image = Image.fromarray(img)
display.display(image)

#### We perform model search on the already trained YOLO5 neural network:

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5x6')

In [ ]:
# Inference
results = model(img)

# Results
positions = results.pandas().xyxy[0]
positions

In [ ]:
results.save()

In [ ]:
# I calculate the centers of each detection
positions["xcenter"] = [(positions["xmax"][i] - positions["xmin"][i])/2 + positions["xmin"][i] for i in range(len(positions))]
positions["ycenter"] = [(positions["ymax"][i] - positions["ymin"][i])/2 + positions["ymin"][i] for i in range(len(positions))]
positions

In [ ]:
# Change the centers from 0-1000 to 100-600 to use the map coordinates in real scene
coords = np.array([[positions["xcenter"][i]+100, positions["ycenter"][i]+100] for i in range(len(positions))])
positions["x"] = coords[:, 0]
positions["y"] = coords[:, 1]
positions

In [ ]:
# Delete all necessary data
detections = positions.drop(columns=["xmin", "ymin", "xmax", "ymax", "confidence", "class", "xcenter", "ycenter"])
detections

### We try to cover the television remote control in the coordinates indicated to us

In [ ]:
# pos20 = [270, 421, 200, 180, 0, 0]
# pos21 = [270, 421, -25, 180, 0, 0] # tv remote coordinates
# robot.set_position2(pos20, True)
# robot.gripper(0)
# time.sleep(2)
# robot.set_position2(pos21, True)
# time.sleep(2)
# robot.gripper(1)
# time.sleep(2)
# robot.set_position2(pos20, True)

# SECTION III - SPEECH RECOGNITION

### To carry out this part you have to install several libraries:


###   - SpeechRecognition:   

pip install SpeechRecognition

###    - PyAudio:

####       Download version according to the version of Python in      https://www.lfd.uci.edu/~gohlke/pythonlibs/#pyaudio

pip install PyAudio-0.2.11-cp(version de Python)-cp...

###    - Googletrans 3.1.0a0

pip install googletrans==3.1.0a0

###     - Install nltk libraries manually (Python 64 bits)

#Run the Python interpreter and type the commands:

#>>> import nltk
#>>> nltk.download()

###    - Install gTTS

pip install gTTS
pip3 install text2digits

## Once installed we import the libraries:

In [ ]:
import re # Necessary for re.sub
import time
import numpy as np
import math
import random
import speech_recognition as sr
import nltk
from gtts import gTTS
from playsound import playsound
import multiprocessing
import os
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from googletrans import Translator
from text2digits import text2digits as t2d

In [ ]:
import nltk
# nltk.download('stopwords')

In [ ]:
# nltk.download('averaged_perceptron_tagger')

### Voice recognition and recording function

In [ ]:
def recognize_speech_from_mic(reconocedor, microfono):
    # Prepare the dictionary to return
    respuesta = {
        "success": False,
        "error": None,
        "transcription": None
    }

    # Check that the recognizer and microphone have been declared
    if not isinstance(reconocedor, sr.Recognizer):
        raise TypeError("`recognizer` must be `Recognizer` instance")
    if not isinstance(microfono, sr.Microphone):
        raise TypeError("`microphone` must be `Microphone` instance")

    # Flag that helps us know if audio has been detected or just silence
    audioReconocido = True

    # The microphone is used as a source
    with microfono as fuente:
        print("Speak Now:")
        # Adjust microphone sensitivity to ambient sound
        reconocedor.adjust_for_ambient_noise(fuente, duration=0.5)

        # Try recording microphone sound
        try:
            audio = reconocedor.listen(fuente, timeout=5)
        except OSError:
            audioReconocido = False
            pass
        except sr.WaitTimeoutError:
            audioReconocido = False
            pass

    # If the audio has been extracted...
    if audioReconocido:
        # ...Try a try/except to avoid errors due to not understanding speech or not being able to connect to the API.
        try:
            # Transcribe el texto si puede
            respuesta["transcription"] = reconocedor.recognize_google(audio, language='es-ES')
            respuesta["success"] = True
        except sr.RequestError:
            # If you were unable to connect to the API, we get an error
            respuesta["error"] = "API unavailable"
        except sr.UnknownValueError:
            #If the voice could not be recognized,
            respuesta["error"] = "Unable to recognize speech"

    return respuesta

### Listen and understand orders function

In [ ]:
def escucharOrdenes(fraseGrabada, traductor, traducir = 1):
    # Translate the phrase to preprocess the English phrases (better for stemming and lemmatizing) 
    if traducir:
        fraseTraducida = traducirFrase(fraseGrabada, traductor)
    else:
        fraseTraducida = fraseGrabada

    # Stopwords and lemmatizers are declared
    stop_words = set(stopwords.words('english'))
    lemmatizador = WordNetLemmatizer()

    # There is no need to send_tokenize because the recording captures only 1 phrase -> There is no need to separate phrases.
    # The received phrase is tokenized by words (similar to string.split(), but with more criteria)

    listaPalabras = nltk.word_tokenize(fraseTraducida)
    listaLemmatizador = []

    for palabra in listaPalabras:
        # Lemmatizing is applied: reduction to the generic root
        listaLemmatizador.append(lemmatizador.lemmatize(palabra))

    # Stopwords are eliminated: words with little semantic value
    listaLemmatizador = [palabra for palabra in listaLemmatizador if palabra not in stop_words]

    # Tagging is applied to relate each token to a type of phrase (verb, subject...)
    fraseFinal = nltk.pos_tag(listaLemmatizador)

    return fraseFinal, fraseTraducida

### Function to translate the sentences into English

In [ ]:
def traducirFrase(fraseGrabada, traductor):
    # Capital letters are removed and some punctuation marks are removed
    fraseBase = fraseGrabada.lower()
    fraseBase = re.sub('\[.*?¿\]\%', ' ', fraseBase)
    # The language is detected and if it is Spanish...
    print("Phrase: ", fraseBase)
    idioma = str(traductor.detect(fraseBase).lang)

    print(idioma)

    # ...the phrase is translated into English
    if (idioma == 'en'):
        fraseTraducida = fraseBase
    else:
        fraseBase = traductor.translate(fraseBase, src='es')
        fraseTraducida = fraseBase.text

    return fraseTraducida

### Function to get only the words with value

In [ ]:
def recuperarOrden(fraseGrabada, traductor):
    
    fraseFinal, fraseTraducida = escucharOrdenes(fraseGrabada, traductor)

    # Words it detects + response phrases for those keywords
    # These words do not match and serve to identify different orders
    MOVER_OBJETO = ("move", "box", "object", "coge")
    MOVER_OBJETO_OK = ["Okay, Ahora lo muevo."]

    NO_ENTENDIDO = ["Lo siento, no te he entendido", "I have not understood you well", "Vuelve a repetir, por fa",
                    "¿Puedes repetirlo otra vez?", "Repeat it again, please" ]

    APAGAR = ("close", "down", "night", "evening", "goodbye", "bye")
    APAGAR_OK = ["Goodbye", "Hasta la vista, beiby", "Adiós", "bye bye"]

    print("")
    print("Frase traducida :", fraseTraducida)
    print("Frase final :", fraseFinal)
    print()

    orden = 0

    '''
    If some of the words in the phrase contain a keyword, it returns a command.
    The order of the if's is important because there are words more relevant than others
    
    Listed orders:
        - 0: Did not understand (default)
        - 1: Move an object that has your arm in front of it
        - 2: Turn off
    '''
    
    fraseRespuesta = 'Lo siento, no te he entendido'

    for palabra in fraseFinal:

        if palabra[0].lower() in MOVER_OBJETO:
            fraseRespuesta = random.choice(MOVER_OBJETO_OK)
            orden = 1
            break

        elif palabra[0].lower() in APAGAR:
             fraseRespuesta = random.choice(APAGAR_OK)
             orden = 2
             break
        else:
                orden = 0

    # If no match found, returns error
    if (orden == 0):
        fraseRespuesta = random.choice(NO_ENTENDIDO)

    return orden, fraseRespuesta

### Main function to start talking to the robot and send the order

In [ ]:
def main():

    # Initialize the robot simulation
    # Initialize the recognizer, the translator and the microphone
    reconocedor = sr.Recognizer()
    microfono = sr.Microphone()
    traductor = Translator()

    # Phrases that the robot will detect to start or end the program
    SALUDO = ("hola", "ey", "hey", "dias", "okey", "ok", "okay", "day", "roboArm", "robot")
    TERMINAR = ("apagate", "apagar", "adios", "close", "luego", "off")
    OTRAS = ("google", "guguel", "siri", "alexa")

    # If after 3 attempts it does not understand any command (start/end program) it automatically closes
    intentos = 0

    # RoboArm is presented
    texto_leer = "Hola soy RoboArm"
    print(texto_leer)
    reproducir_texto_tts(texto_leer)


   # Main code loop. Here the program can access its basic functions
    while True:
       # If RoboArm detects more than 3 failed attempts, it turns off
        if intentos >= 3:
            break
        intentos += 1

        # Listen to what the user says and use a function to tokenize, lemmatize and tag the phrase
        ## Error  resultado = recognize_voice.recognize_speech_from_mic(reconocedor, microfono)
        resultado = recognize_speech_from_mic(reconocedor, microfono)
        
        if resultado['transcription'] is not None:
            resultado, _ = escucharOrdenes(resultado['transcription'], traductor, 0)

        print(resultado)
        # Flag to know whether to start or end the program
        iniciarPrograma = 0

        # If a word is said in the SALUDO/TERMINAR/OTRAS list, an order is started
        for palabra in reversed(resultado):
            aux = palabra[0].lower()
            if aux in SALUDO:
                iniciarPrograma = 1
                break
            elif aux in TERMINAR:
                iniciarPrograma = -1
                break
            elif aux in OTRAS:
                iniciarPrograma = -2
                break

        if iniciarPrograma == 1:
            # RoboArm ofrece sus servicios
            texto_leer = "¿En qué te puedo ayudar?"
            print(texto_leer)
            reproducir_texto_tts(texto_leer)
            intentos = 0

            # The program starts because a greeting has been detected
            while iniciarPrograma == 1:
                if intentos >= 3:
                    break

                # The user sends his order and is recognized by the voice recognizer (translated into English)
                resultado = recognize_speech_from_mic(reconocedor, microfono)
                print(resultado)
                print("Result[succes] = ", resultado['success'])

               # If an error occurs due to not being able to reach the API or due to a capture error, nothing is done
                if resultado['success'] == False:
                    intentos += 1
                    error()
                elif resultado['success'] == True and resultado['transcription'] is None:
                    intentos += 1
                    error()
                else:
                    # If the recognition is positive, it is sent to the decision algorithm and it returns an order and
                    # a number of pages to scroll
                    orden, texto_leer = recuperarOrden(resultado['transcription'], traductor)

                    # If the robot has understood the command, it restarts the attempts and responds
                    if orden != 0:
                        intentos = 0
                        print(texto_leer)
                        reproducir_texto_tts(texto_leer)


                    # If it has not understood the command, it responds, but does not restart attempts
                    elif intentos < 2:
                        print(texto_leer)
                        reproducir_texto_tts(texto_leer)


                    # The robot has not understood the order
                    if orden == 0:
                        intentos += 1

                    # Move object
                    elif orden == 1:
                        move_box()
                        aviso = "Ya lo he movido "
                        reproducir_texto_tts(aviso)

                    # Turn off Robot
                    else:
                        iniciarPrograma = -1
                        break

        # Turn off the program
        if iniciarPrograma == -1:
            break

        # He has not understood/heard anything
        elif iniciarPrograma == 0:
            error()

    # If it has been tried more than 3 times, the robot turns off automatically
    if intentos >= 3:
        texto_leer = "Lo siento no nos entendemos "
        print(texto_leer)
        reproducir_texto_tts(texto_leer)

    # RoboArm se despide
    texto_leer = "Adios"
    print(texto_leer)
    reproducir_texto_tts(texto_leer)


In [ ]:
def error():

    texto_leer = "No te he entendido"
    print(texto_leer)
    reproducir_texto_tts(texto_leer)

In [ ]:
def reproducir_texto_tts(texto_leer, esPagina = False):
    # The sound that would be made when pronouncing the string is created
    tts = gTTS(texto_leer, lang='es-us')

    # The file name is declared and the tts sound is saved
    
    NOMBRE_ARCHIVO = 'file.mp3'
    tts.save(NOMBRE_ARCHIVO)

    # If TTS is going to be used to read the text on a page, there are extra functions
    if esPagina:
        # We use the class recognize_order and recognize_voice, so we declare the necessary variables
        mic = sr.Microphone()
        recog = sr.Recognizer()
        traductor = recognize_order.Translator()

        # We declare a series of keywords to detect
        PARAR = ("para", "parar", "un momento", "espera", "robot", "rider")

        # We declare and initialize an extra thread
        p = multiprocessing.Process(target=playsound, args=(NOMBRE_ARCHIVO,))
        p.start()

        # Flag to end the thread's work if given the command
        ordenParar = False

        # The program ends if the text finishes dictation or if it is ordered to stop
        while p.exitcode == None and ordenParar == False:
            print("El exit code: ", p.exitcode)
            # The words that the user says are collected and passed through NLP
            frase = recognize_speech_from_mic(recog, mic)
            if frase['transcription'] is not None:
                resultado, _ = recognize_order.escucharOrdenes(frase['transcription'], traductor, 0)

                # Check if any word in the sentence matches the stop array
                for palabra in resultado:
                   # If this is the case, set the boolean to True so that it ends and the thread and the for loop end
                    if palabra[0].lower() in PARAR:
                        ordenParar = True
                        break

            elif p.exitcode == 0:
                break

       # End the thread and delete the file
        p.terminate()
        os.remove(NOMBRE_ARCHIVO)

        return ordenParar

   # If TTS is not used to read a page, simply play the sound
    else:
        # NOMBRE_ARCHIVO = os.path.dirname(r"C:\Users\polme\OneDrive - UAB\Escritorio\Universitat\3r Carrera\1r Semestre\Intelligent Robots\Actividades Carlos") + '\\' + NOMBRE_ARCHIVO
        playsound(NOMBRE_ARCHIVO)
        os.remove(NOMBRE_ARCHIVO)


In [ ]:
# main()
# hola --> hola 
# En que te puedo ayudar  --> mover objeto  o move the box,
# Ok ahora lo muevo -- Ya lo he movido

### OBJECTIVE OF THE EXERCISE

THE MAIN OBJECTIVE IS TO BE ABLE TO USE THE REVERSE KINEMATICS TOOLS OF THE UR5 ROBOT TO BE ABLE TO EASILY PICK UP OBJECTS. THE CLASSIFICATION AND DETERMINATION OF THE POSITION AND ORIENTATION OF THOSE OBJECTS MUST BE DONE WITH THE YOLOV5. ALSO YOU MUST USE THE TOOLS WITH THE VOICE RECOGNITION MODE.

THE PROGRAM MUST BE MODIFIED SO THAT THE ROBOT IS ABLE TO UNDERSTAND YOU AND RECEIVE THE FOLLOWING ORDERS:

PICK APPLE
PICK ORANGE
GRAB SCISSORS*
PICK UP TV REMOTE*

*KEEP INTO ACCOUNT THE INCLINATIONS OF THE OBJECTS.


ONCE THE ORDER IS RECEIVED, THE ROBOT MUST TRY TO PICK UP THE ITEM YOU HAVE ASKED FOR FOR A FEW SECONDS (KEEP IN MIND THAT THE PIECE WILL FALL AGAIN SECONDS LATER AND WILL CHANGE ITS POSITION.

YOU CAN IMPROVE THE PROGRAM adding new instructions, new languages...

---
---
---

---

In [ ]:
def recuperarOrden(fraseGrabada, traductor):
    
    # Function to process the recorded phrase and translate it
    fraseFinal, fraseTraducida = escucharOrdenes(fraseGrabada, traductor)

    # Words it detects + response phrases for those keywords
    # These words do not match and serve to identify different orders
    MOVER_OBJETO = ("move", "box", "object", "coge", "orange", "apple", "scissors", "remote") # ADD OBJECTS
    MOVER_OBJETO_OK = ["Okay, Ahora lo muevo."]

    NO_ENTENDIDO = ["Lo siento, no te he entendido", "I have not understood you well", "Vuelve a repetir, por fa",
                    "¿Puedes repetirlo otra vez?", "Repeat it again, please" ]

    APAGAR = ("close", "down", "night", "evening", "goodbye", "bye")
    APAGAR_OK = ["Goodbye", "Hasta la vista, beiby", "Adiós", "bye bye"]

    orden = 0

    fraseRespuesta = 'Lo siento, no te he entendido'

    # Loop through each word in the processed phrase
    for palabra in fraseFinal:
        # Check if the word matches any of the keywords for moving an object
        if palabra[0].lower() in MOVER_OBJETO:
            fraseRespuesta = random.choice(MOVER_OBJETO_OK)
            orden = 1
            break

        # Check if the word matches any of the keywords for turning off
        elif palabra[0].lower() in APAGAR:
             fraseRespuesta = random.choice(APAGAR_OK)
             orden = 2
             break
        else:
                orden = 0

    # If no match found, returns error
    if (orden == 0):
        fraseRespuesta = random.choice(NO_ENTENDIDO)

    return orden, fraseRespuesta

I'll start by modifying the function that handles moving objects so it can precisely locate the object being asked for. Like, if I say 'Coje la manzana', it should pick up the apple, not the remote control or something else.

I'll kick things off by coding a function specifically for grabbing the requested item. After that, I'll set up another function to catch this request through voice commands. This way, the program will grab exactly what I'm asking for.

In [ ]:
# Function to grab the requested object
def grabRequestedObject(model, camera, objeto:str=None):
    crop_img = None
    
    # Check if the object is None
    if objeto is None:
        print("Object not in table.")
    else:
        # Get the image from the camera
        image,resolution=ur5.get_image(camera)
        img=np.array(image).astype(np.uint8)
        img.resize([resolution[1],resolution[0],3])
        img = img[::-1, ::-1, :]
        
        image = Image.fromarray(img)

        # Inference using the YOLOv5 model
        detections = model(img)
        detections.save()
    
        # Extract the positions of the detected objects
        positions = detections.pandas().xyxy[0]
        
        # Calculate the centers of each detection
        positions["xcenter"] = [(positions["xmax"][i] - positions["xmin"][i])/2 + positions["xmin"][i] for i in range(len(positions))]
        positions["ycenter"] = [(positions["ymax"][i] - positions["ymin"][i])/2 + positions["ymin"][i] for i in range(len(positions))]

        # Iterate through the positions to find the requested object
        for pos in positions.itertuples():
            item = pos[-3]
            if item == objeto:
                x, y, xmax, ymax = int(pos[1]), int(pos[2]), int(pos[3]), int(pos[4])
                crop_img = img[y:ymax, x:xmax, :]
                break

        # Change the centers from 0-1000 to 100-600 to use the map coordinates in real scene
        coords = np.array([[positions["xcenter"][i]+100, positions["ycenter"][i]+100] for i in range(len(positions))])
        positions["x"] = coords[:, 0]
        positions["y"] = coords[:, 1]

        # Delete unnecessary data
        detections = positions.drop(columns=["xmin", "ymin", "xmax", "ymax", "confidence", "class", "xcenter", "ycenter"])
        
        # Get the coordinates of the requested object
        for detection in detections.itertuples():
            if detection[1] == objeto:
                x = detection[2]
                y = detection[3]

        # Get the orientation of the object
        orientation = getOrientation(crop_img)

        # Define the positions for the robot to move
        position_zero = [0, 0, 200, 0, 0, 0]
        upper_position = [x, y, 200, 180, 0, orientation]
        grab_position  = [x, y, -35, 180, 0, orientation]
        
        # Move the robot to the upper position
        robot.set_position2(upper_position, True)
        robot.gripper(0)
        time.sleep(2)
        
        # Move the robot to the grab position and close the gripper
        robot.set_position2(grab_position, True)
        time.sleep(3)
        robot.gripper(1)
        time.sleep(3)
        
        # Move the robot back to the upper position and open the gripper
        robot.set_position2(upper_position, True)
        time.sleep(3)
        robot.gripper(0)

I plan to determine the orientation of detected images using certain methods in OpenCV. Although the results are not always highly consistent, these methods generally perform the task satisfactorily.

In [ ]:
def getOrientation(img:np.ndarray):
    angles_list = []
    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Convert image to binary
    _, bw = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    
    # Find all the contours in the thresholded image
    contours, _ = cv2.findContours(bw, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
    for i, c in enumerate(contours):
        # Calculate the area of each contour
        area = cv2.contourArea(c)
      
        # Ignore contours that are too small or too large
        if area < 0 or 100000 < area:
          continue
      
        rect = cv2.minAreaRect(c)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
      
        # Retrieve the key parameters of the rotated bounding box
        center = (int(rect[0][0]),int(rect[0][1])) 
        width = int(rect[1][0])
        height = int(rect[1][1])
        angle = int(rect[2])
          
        if width < height:
          angle = 90 - angle
        else:
          angle = -angle
          
        angles_list.append(angle)
    return min(angles_list)

So, the next step is to get the program to recognize my voice and figure out which item to spot. I'm going to tweak the function Carlos created and add in the specific objects I want it to grab. This way, the program should be able to pick up exactly what I'm asking for.

In [ ]:
def recuperarOrden(fraseGrabada, traductor):
    
    fraseFinal, fraseTraducida = escucharOrdenes(fraseGrabada, traductor)

    # Words it detects + response phrases for those keywords
    # These words do not match and serve to identify different orders
    MOVER_OBJETO = ("move", "box", "object", "coge")
    ELEGIR_OBJETO = ("remote", "scissors", "apple", "orange")
    MOVER_OBJETO_OK = ["Okay, Ahora lo muevo."]

    NO_ENTENDIDO = ["Lo siento, no te he entendido", "I have not understood you well", "Vuelve a repetir, por fa",
                    "¿Puedes repetirlo otra vez?", "Repeat it again, please" ]

    APAGAR = ("close", "down", "night", "evening", "goodbye", "bye")
    APAGAR_OK = ["Goodbye", "Hasta la vista, beiby", "Adiós", "bye bye"]

    orden = 0

    fraseRespuesta = 'No te he entendido'

    for palabra in fraseFinal:
        if palabra[0].lower() in MOVER_OBJETO:
            fraseRespuesta = random.choice(MOVER_OBJETO_OK)
            orden = 1
            break
            
        elif palabra[0].lower() in ELEGIR_OBJETO:
            fraseRespuesta = random.choice(MOVER_OBJETO_OK)
            orden = 1
            objeto = palabra[0].lower()
            break

        elif palabra[0].lower() in APAGAR:
             objeto = None
             fraseRespuesta = random.choice(APAGAR_OK)
             orden = 2
             break
        else:
                orden = 0

    # If no match found, returns error
    if (orden == 0):
        fraseRespuesta = random.choice(NO_ENTENDIDO)
        objeto = None

    return orden, fraseRespuesta, objeto

Call the main function.

In [ ]:
def main(model, camera):
    # Initialize the robot simulation
    # Initialize the recognizer, the translator and the microphone
    reconocedor = sr.Recognizer()
    microfono = sr.Microphone()
    ## Error  traductor = googletrans.Translator()
    traductor = Translator()

    # Phrases that the robot will detect to start or end the program
    SALUDO = ("hola", "ey", "hey", "dias", "okey", "ok", "okay", "day", "roboArm", "robot")
    TERMINAR = ("apagate", "apagar", "adios", "close", "luego", "off")
    OTRAS = ("google", "guguel", "siri", "alexa")

    # If after 3 attempts it does not understand any command (start/end program) it automatically closes
    intentos = 0

    # RoboArm is presented
    texto_leer = "Hola soy RoboArm"
    reproducir_texto_tts(texto_leer)


   # Main code loop. Here the program can access its basic functions
    while True:
       # If RoboArm detects more than 3 failed attempts, it turns off
        if intentos >= 3:
            break
        intentos += 1

        # Listen to what the user says and use a function to tokenize, lemmatize and tag the phrase
        ## Error  resultado = recognize_voice.recognize_speech_from_mic(reconocedor, microfono)
        resultado = recognize_speech_from_mic(reconocedor, microfono)
        
        if resultado['transcription'] is not None:
            resultado, _ = escucharOrdenes(resultado['transcription'], traductor, 0)

        # Flag to know whether to start or end the program
        iniciarPrograma = 0


        # If a word is said in the SALUDO/TERMINAR/OTRAS list, an order is started
        for palabra in reversed(resultado):
            aux = palabra[0].lower()
            if aux in SALUDO:
                iniciarPrograma = 1
                break
            elif aux in TERMINAR:
                iniciarPrograma = -1
                break
            elif aux in OTRAS:
                iniciarPrograma = -2
                break

        if iniciarPrograma == 1:
            # RoboArm ofrece sus servicios
            texto_leer = "¿En qué te puedo ayudar?"
            reproducir_texto_tts(texto_leer)
            intentos = 0

            # The program starts because a greeting has been detected
            while iniciarPrograma == 1:
                if intentos >= 3:
                    break

                # The user sends his order and is recognized by the voice recognizer (translated into English)
                resultado = recognize_speech_from_mic(reconocedor, microfono)

               # If an error occurs due to not being able to reach the API or due to a capture error, nothing is done
                if resultado['success'] == False:
                    intentos += 1
                    error()
                elif resultado['success'] == True and resultado['transcription'] is None:
                    intentos += 1
                    error()
                else:
                    # If the recognition is positive, it is sent to the decision algorithm and it returns an order and
                    # a number of pages to scroll
                    orden, texto_leer, objeto = recuperarOrden(resultado['transcription'], traductor)

                    # If the robot has understood the command, it restarts the attempts and responds
                    if orden != 0:
                        intentos = 0
                        reproducir_texto_tts(texto_leer)

                    # If it has not understood the command, it responds, but does not restart attempts
                    elif intentos < 2:
                        reproducir_texto_tts(texto_leer)

                    # The robot has not understood the order
                    if orden == 0:
                        intentos += 1

                    # Move object
                    elif orden == 1:
                        grabRequestedObject(model, camera, objeto)
                        aviso = "Ya lo he movido "
                        reproducir_texto_tts(aviso)

                    # Turn off Robot
                    else:
                        iniciarPrograma = -1
                        break

        # Turn off the program
        if iniciarPrograma == -1:
            break

        # He has not understood/heard anything
        elif iniciarPrograma == 0:
            error()

    # If it has been tried more than 3 times, the robot turns off automatically
    if intentos >= 3:
        texto_leer = "Lo siento no nos entendemos "
        reproducir_texto_tts(texto_leer)

    # RoboArm se despide
    texto_leer = "Adios"
    reproducir_texto_tts(texto_leer)

In [ ]:
main(model, camera)